In [1]:
import sys 
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [2]:
import time
import copy
import json
import argparse

from pathlib import Path
from pprint import pprint
from importlib.machinery import SourceFileLoader
from importlib import reload

import torch
from torch.utils.data import DataLoader


from src.datasets import TrainMouseVideoDataset, ValMouseVideoDataset, ConcatMiceVideoDataset
from src.utils import get_lr, init_weights, get_best_model_path
from src.responses import get_responses_processor
# from src.ema import ModelEma, EmaCheckpoint
from src.inputs import get_inputs_processor
# from src.metrics import CorrelationMetric
from src.indexes import IndexesGenerator
# from src.argus_models import MouseModel
from src.data import get_mouse_data
from src.mixers import CutMix
from src import constants

from configs.true_batch_ssm_001 import config

In [ ]:
# Initialise the dataloaders
# Define the model 
# Define the loss function 
# Define the optimizer 
# Define the training loop 

In [3]:
argus_params = config["argus_params"]
folds_splits = constants.folds_splits

for fold_split in folds_splits:
    fold_experiment_dir = constants.experiments_dir / "true_batch_001"

    val_folds_splits = [fold_split]
    train_folds_splits = sorted(set(constants.folds_splits) - set(val_folds_splits))

    print(f"Val fold: {val_folds_splits}, train folds: {train_folds_splits}")
    print(f"Fold experiment dir: {fold_experiment_dir}")

Val fold: ['fold_0'], train folds: ['fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_5', 'fold_6']
Fold experiment dir: ../data/experiments/true_batch_001
Val fold: ['fold_1'], train folds: ['fold_0', 'fold_2', 'fold_3', 'fold_4', 'fold_5', 'fold_6']
Fold experiment dir: ../data/experiments/true_batch_001
Val fold: ['fold_2'], train folds: ['fold_0', 'fold_1', 'fold_3', 'fold_4', 'fold_5', 'fold_6']
Fold experiment dir: ../data/experiments/true_batch_001
Val fold: ['fold_3'], train folds: ['fold_0', 'fold_1', 'fold_2', 'fold_4', 'fold_5', 'fold_6']
Fold experiment dir: ../data/experiments/true_batch_001
Val fold: ['fold_4'], train folds: ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_5', 'fold_6']
Fold experiment dir: ../data/experiments/true_batch_001
Val fold: ['fold_5'], train folds: ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_6']
Fold experiment dir: ../data/experiments/true_batch_001
Val fold: ['fold_6'], train folds: ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4', 'f

In [4]:
indexes_generator = IndexesGenerator(**argus_params["frame_stack"])
inputs_processor = get_inputs_processor(*argus_params["inputs_processor"])
responses_processor = get_responses_processor(*argus_params["responses_processor"])

cutmix = CutMix(**config["cutmix"])
train_datasets = []
mouse_epoch_size = config["train_epoch_size"] // constants.num_mice
for mouse in constants.mice:
    train_datasets += [
        TrainMouseVideoDataset(
            mouse_data=get_mouse_data(mouse=mouse, splits=train_folds_splits),
            indexes_generator=indexes_generator,
            inputs_processor=inputs_processor,
            responses_processor=responses_processor,
            epoch_size=mouse_epoch_size,
            mixer=cutmix,
        )
    ]
train_dataset = ConcatMiceVideoDataset(train_datasets)
print("Train dataset len:", len(train_dataset))
val_datasets = []
for mouse in constants.mice:
    val_datasets += [
        ValMouseVideoDataset(
            mouse_data=get_mouse_data(mouse=mouse, splits=val_folds_splits),
            indexes_generator=indexes_generator,
            inputs_processor=inputs_processor,
            responses_processor=responses_processor,
        )
    ]
val_dataset = ConcatMiceVideoDataset(val_datasets)
print("Val dataset len:", len(val_dataset))

train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    num_workers=config["num_dataloader_workers"],
    shuffle=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=config["batch_size"] // argus_params["iter_size"],
    num_workers=config["num_dataloader_workers"],
    shuffle=False,
)

Train dataset len: 72000
Val dataset len: 585


In [5]:
from src.models.dwiseneurossm import DwiseNeuroSSM
argus_params = config["argus_params"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DwiseNeuroSSM(**argus_params["nn_module"][1])
model = model.to(device)

/workspace/sensorium-ssm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for batch in train_loader:
    inputs, target = batch 
    inputs = inputs.to(device)
    result = model(inputs)
    break

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 47.50 GiB of which 72.31 MiB is free. Process 3714451 has 47.42 GiB memory in use. Of the allocated memory 46.80 GiB is allocated by PyTorch, and 140.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
torch.cuda.empty_cache()